# Phase 6: Results Summary & Production Deployment

This notebook summarizes all findings and provides production deployment recommendations.

## Project Overview
Built a causal ML system that:
1. Predicts customer churn (LightGBM, AUC ~0.82)
2. Estimates causal intervention effects (Doubly Robust, ATE ~-12%)
3. Identifies heterogeneous effects (Causal Forest)
4. Validates with A/B testing (24% relative churn reduction)

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from pathlib import Path
import sys
sys.path.append('..')

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 1. Load All Results

In [ ]:
# Load final dataset
df = pd.read_csv('../data/processed/churn_with_hte.csv')
print(f"Dataset Shape: {df.shape}")

# Load models
models_dir = Path('../models')
model_files = list(models_dir.glob('*.pkl'))
print(f"\nSaved models: {[f.name for f in model_files]}")

## 2. Executive Summary

In [ ]:
print("=" * 70)
print("EXECUTIVE SUMMARY")
print("=" * 70)

print("""
PROJECT: Churn Prediction with Heterogeneous Treatment Effects

PROBLEM: Traditional churn models predict WHO churns, but not WHAT intervention works.

SOLUTION: Causal ML system that:
  1. Predicts churn probability (who is at risk)
  2. Estimates causal intervention effects (does intervention work?)
  3. Identifies heterogeneous effects (who responds best?)
  4. Recommends personalized interventions (optimal action per customer)

KEY RESULTS:
  - Churn Prediction AUC: 0.82 (good separation)
  - Average Treatment Effect: -12% (intervention reduces churn by 12pp)
  - A/B Test Validation: 24% relative churn reduction (p < 0.001)
  - ROI: Positive (net benefit per 1000 customers: $45,000+)

BUSINESS IMPACT:
  - Without intervention: ~42% churn in at-risk population
  - With intervention: ~32% churn (10pp reduction)
  - Customers saved: ~100 per 1000 at-risk
  - Revenue protected: Significant (varies by LTV)
""")

## 3. Key Metrics Dashboard

In [ ]:
# Create metrics summary
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Churn Distribution
churn_counts = df['churn_observed'].value_counts()
axes[0, 0].pie(churn_counts, labels=['Retained', 'Churned'], autopct='%1.1f%%',
               colors=['#2ecc71', '#e74c3c'], explode=(0, 0.05))
axes[0, 0].set_title('Overall Churn Distribution', fontsize=14, fontweight='bold')

# 2. Treatment Effect Distribution
axes[0, 1].hist(df['treatment_effect'], bins=40, color='#3498db', edgecolor='white', alpha=0.7)
axes[0, 1].axvline(x=df['treatment_effect'].mean(), color='red', linestyle='--', linewidth=2)
axes[0, 1].axvline(x=0, color='black', linestyle='-', linewidth=1)
axes[0, 1].set_xlabel('Treatment Effect')
axes[0, 1].set_ylabel('Count')
axes[0, 1].set_title('Heterogeneous Treatment Effects', fontsize=14, fontweight='bold')

# 3. Effect by Tenure
if 'tenure_segment' not in df.columns:
    df['tenure_segment'] = pd.cut(df['tenure'], 
                                   bins=[0, 6, 12, 24, 48, 72],
                                   labels=['0-6m', '6-12m', '12-24m', '24-48m', '48+m'])
tenure_effects = df.groupby('tenure_segment')['treatment_effect'].mean()
tenure_effects.plot(kind='bar', ax=axes[1, 0], color='#9b59b6')
axes[1, 0].set_xlabel('Tenure Segment')
axes[1, 0].set_ylabel('Mean Treatment Effect')
axes[1, 0].set_title('Treatment Effect by Tenure', fontsize=14, fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# 4. Model Performance Metrics
metrics = ['AUC-ROC', 'ATE', 'Churn Reduction']
values = [0.82, 0.12, 0.24]
colors = ['#3498db', '#2ecc71', '#e67e22']
bars = axes[1, 1].barh(metrics, values, color=colors)
axes[1, 1].set_xlabel('Value')
axes[1, 1].set_title('Key Performance Metrics', fontsize=14, fontweight='bold')
for bar, val in zip(bars, values):
    axes[1, 1].text(val + 0.02, bar.get_y() + bar.get_height()/2, f'{val:.2f}', 
                    va='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../docs/results_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Segment Recommendations

In [ ]:
print("=" * 70)
print("INTERVENTION RECOMMENDATIONS BY SEGMENT")
print("=" * 70)

# Segment analysis
segments = [
    ('New Customers (0-6 months)', 'new_customer == 1'),
    ('Month-to-Month Contracts', 'Contract == "Month-to-month"'),
    ('Fiber Optic Internet', 'InternetService == "Fiber optic"'),
    ('High-Value Customers', 'high_value == 1'),
    ('Long-Tenure (24+ months)', 'long_tenure == 1')
]

print(f"\n{'Segment':<40} {'Count':>8} {'Churn %':>10} {'Avg Effect':>12} {'Priority':>10}")
print("-" * 80)

for name, query in segments:
    try:
        subset = df.query(query)
        count = len(subset)
        churn_rate = subset['churn_observed'].mean() * 100
        avg_effect = subset['treatment_effect'].mean()
        
        # Priority based on effect magnitude and churn rate
        if avg_effect < -0.10 and churn_rate > 30:
            priority = 'HIGH'
        elif avg_effect < -0.05:
            priority = 'MEDIUM'
        else:
            priority = 'LOW'
        
        print(f"{name:<40} {count:>8} {churn_rate:>9.1f}% {avg_effect:>11.3f} {priority:>10}")
    except:
        pass

print("\n" + "-" * 70)
print("PRIORITY KEY:")
print("  HIGH   = Strong intervention effect + high churn risk")
print("  MEDIUM = Moderate intervention effect")
print("  LOW    = Weak effect (save budget)")
print("-" * 70)

## 5. Production Deployment Architecture

In [ ]:
print("=" * 70)
print("PRODUCTION SYSTEM ARCHITECTURE")
print("=" * 70)

print("""
                    +---------------------------+
                    |  Customer Trigger Event   |
                    |  (login, billing, etc.)   |
                    +------------+--------------+
                                 |
                                 v
                    +---------------------------+
                    |   1. Fetch Customer Data  |
                    |   - Tenure, MRR, usage    |
                    |   - Recent behavior       |
                    +------------+--------------+
                                 |
                                 v
                    +---------------------------+
                    |   2. Churn Risk Scoring   |
                    |   - LightGBM model        |
                    |   - If risk > 30%, cont.  |
                    +------------+--------------+
                                 |
                                 v
                    +---------------------------+
                    |  3. Treatment Effect Est. |
                    |  - Causal Forest          |
                    |  - Per-intervention CATE  |
                    +------------+--------------+
                                 |
                                 v
                    +---------------------------+
                    |  4. Intervention Select   |
                    |  - Choose best option     |
                    |  - Consider cost/benefit  |
                    +------------+--------------+
                                 |
                                 v
                    +---------------------------+
                    |  5. Execute & Log         |
                    |  - Trigger intervention   |
                    |  - Record for monitoring  |
                    +---------------------------+

API ENDPOINTS:
  POST /predict         - Get intervention recommendation
  POST /batch-predict   - Score multiple customers
  GET  /health          - Health check
""")

## 6. Monitoring & Maintenance Plan

In [ ]:
print("=" * 70)
print("MONITORING & MAINTENANCE PLAN")
print("=" * 70)

print("""
1. MODEL PERFORMANCE MONITORING
   - Track churn prediction AUC weekly
   - Alert if AUC drops below 0.75
   - Compare predicted effects to actual outcomes monthly

2. DATA DRIFT DETECTION
   - KS test on feature distributions weekly
   - Alert if p-value < 0.01 for any feature
   - Trigger retraining if multiple features drift

3. INTERVENTION EFFECTIVENESS
   - Track actual churn rate by intervention type
   - Run periodic A/B tests (quarterly)
   - Adjust intervention mix based on results

4. RETRAINING SCHEDULE
   - Churn model: Monthly on rolling 12-month data
   - Causal forest: Quarterly with new intervention data
   - Feature engineering: As-needed based on drift alerts

5. COST MONITORING
   - Track intervention costs by type
   - Monitor ROI per segment
   - Optimize budget allocation quarterly
""")

## 7. Future Improvements

In [ ]:
print("=" * 70)
print("FUTURE IMPROVEMENTS")
print("=" * 70)

print("""
SHORT TERM (Next Quarter):
  1. Add more intervention types (feature upsell, dedicated support)
  2. Implement real-time feature computation
  3. Add confidence intervals to API responses

MEDIUM TERM (6 Months):
  1. Multi-armed bandit for continuous intervention optimization
  2. Dynamic pricing integration (personalized discount amounts)
  3. Expand to cross-sell/upsell recommendations

LONG TERM (1 Year):
  1. Sequential decision making (multiple touch-points)
  2. Reinforcement learning for intervention timing
  3. Integration with CRM for automated execution

RESEARCH DIRECTIONS:
  1. Sensitivity analysis for unmeasured confounders
  2. Instrumental variable approaches for stronger causal claims
  3. Transfer learning across customer segments
""")

## Summary

### What We Built
A complete causal ML pipeline for churn prediction and intervention optimization:

1. **EDA** (Notebook 01): Identified key churn patterns and risk factors
2. **Churn Model** (Notebook 02): LightGBM with AUC 0.82
3. **Causal Inference** (Notebook 03): Doubly robust ATE estimation
4. **Heterogeneous Effects** (Notebook 04): Causal forest for personalization
5. **A/B Validation** (Notebook 05): Statistical validation of causal claims
6. **Production Ready** (Notebook 06): Deployment architecture and monitoring

### Key Differentiators
- **Causal, not just predictive**: We estimate WHAT works, not just WHO churns
- **Personalized**: Different interventions for different customers
- **Validated**: A/B testing confirms our causal estimates
- **Production-ready**: Complete API and monitoring design

### Business Impact
- 24% relative churn reduction in at-risk population
- Positive ROI on intervention program
- Personalized recommendations maximize effectiveness